## Caproicproducens

In [1]:
from Bio import Entrez, SeqIO
import pandas as pd
import os

import sys 
sys.path.append("../functions/")

import EGC

### Find ORFs and translate sequences -> fasta file

In [2]:
table = 11
min_pro_len = 100

# https://biopython.org/DIST/docs/tutorial/Tutorial.html#sec443

def find_orfs_with_trans(seq, trans_table, min_protein_length):
    answer = []
    seq_len = len(seq)
    for strand, nuc in [(+1, seq), (-1, seq.reverse_complement())]:
        for frame in range(3):
            trans = nuc[frame:].translate(trans_table)
            trans_len = len(trans)
            aa_start = 0
            aa_end = 0
            while aa_start < trans_len:
                aa_end = trans.find("*", aa_start)
                if aa_end == -1:
                    aa_end = trans_len
                if aa_end - aa_start >= min_protein_length:
                    if strand == 1:
                        start = frame + aa_start * 3
                        end = min(seq_len, frame + aa_end * 3 + 3)
                    else:
                        start = seq_len - frame - aa_end * 3 - 3
                        end = seq_len - frame - aa_start * 3
                    answer.append((start, end, strand, trans[aa_start:aa_end]))
                aa_start = aa_end + 1
    answer.sort()
    return answer

In [3]:
all_recs = []
i = 1

ofile = open("../output/Caproicproducens_CH14_bin_0/CH14-bin.0.faa","w")     
for record in SeqIO.parse("../input/CH14-bin.0.fa","fasta"):
    orf_list = find_orfs_with_trans(record.seq, table, min_pro_len)
    
    for start, end, strand, pro in orf_list:
        ofile.write(">"+"gene"+str(i)+"\n"+str(pro)+"\n")
        i +=1
ofile.close()

/Users/idunmariaburgos/opt/anaconda3/envs/MAGs_RecAndAn2/lib/python3.9/site-packages/Bio/Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


### Acetate transport


- 3.A.1.120.5 - abc
- 1.A.14.2.2 - acetate uptake
- 2.A.23.1.10 - uptake
- 2.A.6.2.9 - abc
- 2.A.96.1.11 - uptake


- 2.A.1.13.1 - proton linked efflux/influx
- 2.A.1.6.11 - Acetate/haloacid transporter (no mechanism)
- 2.A.102.4.13 - acetate transporter
- 2.A.21.7.3 - Pyruvate/acetate/propionate: H+ symporter
- 2.A.96.1.4 - acetate permease

In [4]:
from io import TextIOWrapper
from Bio import SeqIO 
from urllib.request import urlopen
import re

In [5]:
possible_transporters = pd.read_excel("/Users/idunmariaburgos/Documents/Work/Project/chain elongation/Clostridia BL3 and BL4/possible_transporters_tcdb.xlsx",sheet_name="overview")

possible_transporters_regex = "|".join(possible_transporters["Acetate"].values)

**Filter data from tcdb**

In [6]:
handle = TextIOWrapper(urlopen("https://www.tcdb.org/public/tcdb"))
recs = list(SeqIO.parse(handle, 'fasta'))
handle.close()

In [7]:
ofile = open("../output/Caproicproducens_CH14_bin_0/transporters.faa","w")

for rec in recs:
    id_ = str(rec.id)
    
    if re.search(possible_transporters_regex, id_):
        
        
        sequence = str(rec.seq)
        
        ofile.write(">"+id_+"\n"+sequence+"\n")
ofile.close()        


#### Check matches

**No good match** 
- Best matches (F:ATP binding): https://www.tcdb.org/search/result.php?acc=q2pgb8

In [8]:
transporters_CH14_bin_0 = pd.read_csv("../output/Caproicproducens_CH14_bin_0/CH14-bin.0_transporters.tsv",
            sep="\t",
            header=None)

transporters_CH14_bin_0.columns=["query acc.ver", "subject acc.ver", "% identity", "alignment length", "mismatches", "gap opens", "q. start", "q. end", "s. start", "s. end", "evalue", "bit score"]
transporters_CH14_bin_0 = transporters_CH14_bin_0[["query acc.ver","subject acc.ver","% identity","evalue","bit score"]]

In [9]:
transporters_CH14_bin_0.sort_values("bit score",ascending=False).head(10)

,query acc.ver,subject acc.ver,% identity,evalue,bit score
66,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene9840,31.239,4.750000e-84,273.0
68,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene948,27.273,6.300000e-57,200.0
67,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene9840,35.398,7.070000e-33,131.0
69,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene948,34.555,2.580000e-23,102.0
70,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene2822,26.142,1.120000e-22,99.8
73,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene11089,23.387,1.230000e-21,96.3
78,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene12059,24.440,2.630000e-17,82.8
76,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene10468,30.244,8.670000e-18,82.4
81,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene45,23.664,1.350000e-16,80.9
82,gnl|TC-DB|Q2PGB8|3.A.1.120.5,gene4578,23.379,1.430000e-16,80.9


## Create model

### Constraints

In [10]:
soft_constraints = {"R_EX_hxa_e":1,
"R_EX_but_e":1,
"R_EX_ac_e":1,
"R_EX_eoth_e":1,
"R_EX_h2_e":1,
"R_EX_co2_e":1}

In [11]:
pd.Series(soft_constraints).to_csv("../output/Caproicproducens_CH14_bin_0/soft_constraints.tsv",sep="\t",index_label=None,header=None)

## Fixing acetate transport

In [12]:
import reframed
import pandas as pd

import copy

In [13]:
model = reframed.load_cbmodel("../output/Caproicproducens_CH14_bin_0/CH14-bin.0.xml")

media_db = pd.read_csv("https://raw.githubusercontent.com/cdanielmachado/carveme/master/carveme/data/input/media_db.tsv",sep="\t")

In [14]:
model.get_objective()

{'R_Growth': 1.0}

**Model has the wrong acetate transporter**

In [15]:
model.get_metabolite_reactions("M_ac_e")

['R_EX_ac_e', 'R_Acabc']

In [16]:
model.reactions.R_ACKr

R_ACKr: M_ac_c + M_atp_c <-> M_actp_c + M_adp_c

In [17]:
model.reactions.R_PTAr

R_PTAr: M_accoa_c + M_pi_c <-> M_actp_c + M_coa_c

**Removing previous and adding new acetate transporter**

In [18]:
model_uni = reframed.load_cbmodel("../input/universe_bacteria.xml")
model_uni.reactions.R_ACt2r

R_ACt2r: M_ac_e + M_h_e --> M_ac_c + M_h_c

In [19]:
R_ACt2r = copy.copy(model_uni.reactions.R_ACt2r)

model.remove_reaction("R_Acabc")

model.add_reaction(R_ACt2r)

In [20]:

model.reactions.R_ACt2r.reversible=True
model.reactions.R_ACt2r.lb=-1000
model.update()

model.reactions.R_ACt2r.reversible

True

### Test model

In [21]:
LB_env = reframed.Environment.from_compounds(media_db[media_db.medium=="LB[-O2]"].compound.values)

LB_env.apply(model,warning=False,inplace=True)

reframed.pFBA(model).show_values(pattern="R_EX_",sort=True,abstol=1e-2)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-14
R_EX_ala__L_e -10
R_EX_cys__L_e -10
R_EX_glc__D_e -10
R_EX_gsn_e   -10
R_EX_h2o_e   -10
R_EX_so4_e   -10
R_EX_gly_e   -2.01094
R_EX_pi_e    -1.20899
R_EX_glu__L_e -0.768834
R_EX_asp__L_e -0.716604
R_EX_thr__L_e -0.667229
R_EX_leu__L_e -0.552368
R_EX_lys__L_e -0.420728
R_EX_ura_e   -0.405567
R_EX_arg__L_e -0.362652
R_EX_pro__L_e -0.271022
R_EX_hxan_e  -0.250476
R_EX_k_e     -0.239314
R_EX_phe__L_e -0.227142
R_EX_met__L_e -0.188698
R_EX_tyr__L_e -0.169112
R_EX_his__L_e -0.116152
R_EX_trp__L_e -0.0696916
R_EX_mg2_e   -0.0106359
R_EX_but_e    4.62522
R_EX_h_e      4.74912
R_EX_gua_e    9.70234
R_EX_nh4_e    17.9287
R_EX_h2s_e    19.8816
R_EX_h2_e     36.6746
R_EX_ac_e     39.981
R_EX_co2_e    52.3525


In [22]:
reframed.FVA(model,obj_frac=1,reactions=["R_EX_ac_e","R_EX_hxa_e","R_EX_but_e"])

{'R_EX_ac_e': [39.72765181372839, 40.25205685021697],
 'R_EX_hxa_e': [0.0, 3.9945308624399694],
 'R_EX_but_e': [0.6306934267398674, 5.149629325669899]}

#### Model can use lactate

In [23]:
LB_env = reframed.Environment.from_compounds(media_db[media_db.medium=="LB[-O2]"].compound.values)

LB_env.pop("R_EX_glc__D_e")
LB_env["R_EX_lac__L_e"]=(-10,0)
LB_env.apply(model,warning=False,inplace=True)
reframed.pFBA(model).show_values(pattern="R_EX_",sort=True,abstol=1e-2)

R_EX_h_e     -10
R_EX_cys__L_e -10
R_EX_gsn_e   -10
R_EX_h2o_e   -10
R_EX_lac__L_e -10
R_EX_so4_e   -9.73513
R_EX_ala__L_e -9.00093
R_EX_gly_e   -0.870872
R_EX_pi_e    -0.801721
R_EX_glu__L_e -0.509841
R_EX_thr__L_e -0.442463
R_EX_leu__L_e -0.366294
R_EX_lys__L_e -0.279
R_EX_ura_e   -0.268946
R_EX_arg__L_e -0.240487
R_EX_pro__L_e -0.179724
R_EX_hxan_e  -0.166099
R_EX_k_e     -0.158697
R_EX_phe__L_e -0.150626
R_EX_met__L_e -0.125132
R_EX_tyr__L_e -0.112144
R_EX_his__L_e -0.0770247
R_EX_trp__L_e -0.046215
R_EX_but_e    3.06715
R_EX_gua_e    9.80261
R_EX_nh4_e    16.6895
R_EX_h2s_e    19.6566
R_EX_h2_e     23.3033
R_EX_ac_e     33.8962
R_EX_co2_e    41.3937


In [24]:
reframed.FVA(model,obj_frac=1,reactions=["R_EX_ac_e","R_EX_hxa_e","R_EX_but_e"])

{'R_EX_ac_e': [33.89623868670155, 34.596546573222334],
 'R_EX_hxa_e': [0.0, -0.0],
 'R_EX_but_e': [3.0671483091190956, 3.0671483091190526]}

#### Model cannot grow on M9

In [25]:
M9_env = reframed.Environment.from_compounds(media_db[media_db.medium=="M9[-O2]"].compound.values)

M9_env.apply(model,warning=False,inplace=True)

reframed.pFBA(model).show_values(pattern="R_EX_",sort=True,abstol=1e-2)

#### Model grows on gapfill medium

In [26]:
gapfill_media = pd.read_csv("../output/gapfill_media/gapfill_media.tsv",sep="\t")

In [27]:
gapfill_env = reframed.Environment.from_compounds(gapfill_media[gapfill_media.medium=="CD_X"].compound.values)
gapfill_env.apply(model,warning=False,inplace=True)
reframed.pFBA(model).show_values(pattern="R_EX_",sort=True,abstol=1e-2)

R_EX_ascb__L_e -10
R_EX_cys__L_e -10
R_EX_xyl__D_e -10
R_EX_so4_e   -8.06239
R_EX_h2o_e   -1.14268
R_EX_pi_e    -0.725947
R_EX_ura_e   -0.243526
R_EX_gua_e   -0.178733
R_EX_k_e     -0.143698
R_EX_nh4_e    3.13973
R_EX_h_e      12.7415
R_EX_h2s_e    17.878
R_EX_h2_e     23.8928
R_EX_ac_e     32.7424
R_EX_co2_e    44.9839


In [28]:
reframed.FVA(model,obj_frac=1,reactions=["R_EX_ac_e","R_EX_hxa_e","R_EX_but_e"])

{'R_EX_ac_e': [32.742372929457396, 41.59969725107186],
 'R_EX_hxa_e': [0.0, -0.0],
 'R_EX_but_e': [0.0, -1.8271098919545435e-12]}

#### Model does not grow on medium used in simulations

In [29]:
model.set_objective({"R_Growth":1,"R_ATPM":0})
model.get_objective()

{'R_Growth': 1}

In [30]:
media_db_constrained = pd.read_csv("../output/steadiercom_sample_0.1.3/media_db_constrained.tsv",sep="\t")

constrained_medium_env = reframed.Environment.from_compounds(media_db_constrained[media_db_constrained.medium=="SC2_X"].compound.values)

SC1_X = media_db_constrained[media_db_constrained.medium=="SC1_X"].copy()
bound_dict = SC1_X[["compound","bound"]].set_index("compound",drop=True)["bound"].to_dict()

for met, bound in bound_dict.items():
    constrained_medium_env["R_EX_"+met+"_e"]=(-bound,1000)

In [31]:
constrained_medium_env.apply(model,warning=False,inplace=True)
reframed.pFBA(model).show_values(pattern="R_EX_",sort=True)

R_EX_h2o_e   -0.01408
R_EX_cys__L_e -0.01
R_EX_h_e     -0.00138825
R_EX_so4_e   -0.000474832
R_EX_pi_e    -9.18536e-05
R_EX_k_e     -1.8182e-05
R_EX_ascb__L_e -1e-05
R_EX_mg2_e   -8.08068e-07
R_EX_fe3_e   -7.27307e-07
R_EX_fe2_e   -6.2666e-07
R_EX_ca2_e   -4.8484e-07
R_EX_cl_e    -4.8484e-07
R_EX_cu2_e   -6.60426e-08
R_EX_mn2_e   -6.4366e-08
R_EX_pnto__R_e -5.36538e-08
R_EX_ribflv_e -4.15445e-08
R_EX_zn2_e   -3.17638e-08
R_EX_thm_e   -1.72791e-08
R_EX_cobalt2_e -9.31493e-09
R_EX_pheme_e  1.16436e-09
R_EX_5drib_e  2.32872e-09
R_EX_4hba_e   3.49309e-09
R_EX_h2_e     0.0055515
R_EX_ac_e     0.00861186
R_EX_co2_e    0.00886337
R_EX_nh4_e    0.00895727
R_EX_h2s_e    0.0104515


In [32]:
reframed.FVA(model,obj_frac=1,reactions=["R_EX_ac_e","R_EX_hxa_e","R_EX_but_e"])

{'R_EX_ac_e': [0.008611864868608111, 0.008611864868607783],
 'R_EX_hxa_e': [0.0, -0.0],
 'R_EX_but_e': [0.0, -0.0]}

##### SynCon1 with lactate

In [33]:
constrained_medium_env["R_EX_lac__L_e"]=(-10,0)

In [34]:
constrained_medium_env.apply(model,warning=False,inplace=True)
sol = reframed.pFBA(model)
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.09753270309327292
R_EX_lac__L_e -10
R_EX_h2o_e   -5.72448
R_EX_nh4_e   -1.0818
R_EX_h_e     -0.50551
R_EX_pi_e    -0.0961763
R_EX_k_e     -0.0190377
R_EX_so4_e   -0.0144261
R_EX_cys__L_e -0.01
R_EX_ac_e     8.5391
R_EX_co2_e    8.79198
R_EX_h2_e     17.2483


In [35]:
sol = reframed.pFBA(model,objective={"R_ATPM":1,"R_Growth":0})
print(sol.values["R_ATPM"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

10.010033333333332
R_EX_h2o_e   -10.02
R_EX_lac__L_e -10
R_EX_cys__L_e -0.01
R_EX_nh4_e    0.01
R_EX_h2s_e    0.01
R_EX_ac_e     10.01
R_EX_co2_e    10.01
R_EX_h2_e     20.01


In [36]:
sol.show_metabolite_balance("M_fdxrd_c",model,abstol=1e-4,equations=True)

[ --> o ] R_POR_syn     20.02    	M_coa_c + 2.0 M_fdxo_2_2_c + M_pyr_c --> M_accoa_c + M_co2_c + 2.0 M_fdxrd_c + M_h_c
[ o --> ] R_FRROX      -20       	M_nadh_c + 2.0 M_fdxrd_c + M_h_c + 2.0 M_nadp_c --> 2.0 M_fdxo_2_2_c + M_nad_c + 2.0 M_nadph_c
[ <-- o ] R_FNRR3_1    -0.02     	2.0 M_fdxo_2_2_c + M_h2_c <-> 2.0 M_fdxrd_c + 2.0 M_h_c


In [37]:
sol.show_metabolite_balance("M_atp_c",model,abstol=1e-1,equations=True)

[ o <-- ] R_ACKr        10.01    	M_ac_c + M_atp_c <-> M_actp_c + M_adp_c
[ o --> ] R_ATPM       -10.01    	M_atp_c + M_h2o_c --> M_adp_c + M_h_c + M_pi_c


In [38]:
reframed.FVA(model,obj_frac=1,reactions=["R_EX_ac_e","R_EX_hxa_e","R_EX_but_e"])

{'R_EX_ac_e': [8.53910317304766, 8.53910317304766],
 'R_EX_hxa_e': [0.0, -0.0],
 'R_EX_but_e': [0.0, -0.0]}

##### SynCon1 with D-xylose

In [39]:
constrained_medium_env.pop("R_EX_lac__L_e")
constrained_medium_env["R_EX_xyl__D_e"]=(-10,0)

In [40]:
constrained_medium_env.apply(model,warning=False,inplace=True)
sol = reframed.pFBA(model)
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.27556816862128863
R_EX_xyl__D_e -10
R_EX_so4_e   -6.40472
R_EX_nh4_e   -3.07476
R_EX_pi_e    -0.271736
R_EX_k_e     -0.053789
R_EX_cys__L_e -0.01
R_EX_mg2_e   -0.00239055
R_EX_fe3_e   -0.00215164
R_EX_fe2_e   -0.00185388
R_EX_ca2_e   -0.00143433
R_EX_cl_e    -0.00143433
R_EX_h2_e     0.0844972
R_EX_h_e      2.51044
R_EX_h2s_e    6.34571
R_EX_ac_e     12.5208
R_EX_co2_e    13.2352
R_EX_h2o_e    20.9058


In [41]:
sol = reframed.pFBA(model,objective={"R_ATPM":1,"R_Growth":0})
print(sol.values["R_ATPM"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

28.34336666666667
R_EX_xyl__D_e -10
R_EX_so4_e   -5
R_EX_cys__L_e -0.01
R_EX_nh4_e    0.01
R_EX_h2o_e    3.31331
R_EX_h2s_e    5.01
R_EX_h_e      6.66668
R_EX_h2_e     13.3434
R_EX_ac_e     16.6767
R_EX_co2_e    16.6767


In [42]:
sol.show_metabolite_balance("M_fdxrd_c",model,abstol=1e-4,equations=True)

[ --> o ] R_POR_syn     33.3534  	M_coa_c + 2.0 M_fdxo_2_2_c + M_pyr_c --> M_accoa_c + M_co2_c + 2.0 M_fdxrd_c + M_h_c
[ o --> ] R_FRROX      -33.3334  	M_nadh_c + 2.0 M_fdxrd_c + M_h_c + 2.0 M_nadp_c --> 2.0 M_fdxo_2_2_c + M_nad_c + 2.0 M_nadph_c
[ <-- o ] R_FNRR3_1    -0.02     	2.0 M_fdxo_2_2_c + M_h2_c <-> 2.0 M_fdxrd_c + 2.0 M_h_c


In [43]:
sol.show_metabolite_balance("M_atp_c",model,abstol=1e-1,equations=True)

[ --> o ] R_PYK         13.3333  	M_adp_c + M_h_c + M_pep_c --> M_atp_c + M_pyr_c
[ o <-- ] R_ACKr        16.6767  	M_ac_c + M_atp_c <-> M_actp_c + M_adp_c
[ o <-- ] R_PGK         16.6667  	M_3pg_c + M_atp_c <-> M_13dpg_c + M_adp_c
[ o --> ] R_PFK_3      -3.33334  	M_atp_c + M_s7p_c --> M_adp_c + M_h_c + M_s17bp_c
[ o --> ] R_SULabc     -5        	M_atp_c + M_h2o_c + M_so4_e --> M_adp_c + M_h_c + M_pi_c + M_so4_c
[ o --> ] R_XYLK       -10       	M_atp_c + M_xylu__D_c --> M_adp_c + M_h_c + M_xu5p__D_c
[ o --> ] R_ATPM       -28.3434  	M_atp_c + M_h2o_c --> M_adp_c + M_h_c + M_pi_c


### RNF -  H+/Na+-translocating ferredoxin:NAD+ oxidoreductase

In [44]:
from reframed.io.sbml import parse_gpr_rule

In [45]:
model.reactions.R_ATPS4rpp

R_ATPS4rpp: M_adp_c + 4.0 M_h_p + M_pi_c <-> M_atp_c + M_h2o_c + 3.0 M_h_c

**Results from BlastKOALA search (14.08.24) show that RNF-related proteins exist in our MAG**

In [46]:
blast_koala_results = pd.read_table("../output/Caproicproducens_CH14_bin_0/user_ko_definition.txt",header=None)
blast_koala_results = blast_koala_results[[0,1,2]].copy()
blast_koala_results.columns=["gene","KO","description"]
blast_koala_results = blast_koala_results.dropna()

blast_koala_results[blast_koala_results.description.str.contains("rnf")]

,gene,KO,description
966,gene3270,K03616,rnfB; H+/Na+-translocating ferredoxin:NAD+ oxi...
967,gene3275,K03617,rnfA; H+/Na+-translocating ferredoxin:NAD+ oxi...
968,gene3277,K03613,rnfE; H+/Na+-translocating ferredoxin:NAD+ oxi...
969,gene3280,K03612,rnfG; H+/Na+-translocating ferredoxin:NAD+ oxi...
970,gene3282,K03614,rnfD; H+/Na+-translocating ferredoxin:NAD+ oxi...
971,gene3286,K03615,rnfC; H+/Na+-translocating ferredoxin:NAD+ oxi...
2692,gene9189,K03614,rnfD; H+/Na+-translocating ferredoxin:NAD+ oxi...
2693,gene9195,K03613,rnfE; H+/Na+-translocating ferredoxin:NAD+ oxi...


**Adding this protein**

In [47]:
gpr = parse_gpr_rule("G_gene3275 and G_gene3270 and G_gene3286 and (G_gene3282 or G_gene9189) and (G_gene9195 or G_gene3277) and G_gene3280")

In [48]:
R_RnfH2_2 = reframed.CBReaction("R_RnfH2_2",
                    name="energy-converting NADH:ferredoxin oxidoreductase transport of H+",
                   reversible=False,
                   stoichiometry={'M_nad_c':-1,'M_fdxrd_c':-2, 'M_h_c':-2, 'M_nadh_c':1,'M_fdxo_2_2_c':2,'M_h_p':1},
                    gpr_association=gpr
                   )

In [49]:
results = EGC.EGC_identifier(model)

There are NO energy producing cycles in the model


In [50]:
model.add_reaction(R_RnfH2_2)
model.update()

In [51]:
results = EGC.EGC_identifier(model)

There ARE energy producing cycles in the model


In [52]:
results

{'R_ATPM': [('R_ATPS4rpp', 250.0),
  ('R_FNOR', -1000.0),
  ('R_G3PD1ir', 1000.0),
  ('R_G3PD2', 1000.0),
  ('R_ATPM', 250.0),
  ('R_RnfH2_2', 1000.0)],
 'R_CTPM': [('R_ALCD4', -1000.0),
  ('R_ALCD4y', 1000.0),
  ('R_ATPS4rpp', 250.0),
  ('R_FNOR', -1000.0),
  ('R_NDPK3', 250.0),
  ('R_RnfH2_2', 1000.0),
  ('R_CTPM', 250.0)],
 'R_GTPM': [('R_ATPS4rpp', 250.0),
  ('R_FNOR', -1000.0),
  ('R_GLUDxi', -1000.0),
  ('R_GLUDy', 1000.0),
  ('R_PPK50_B', 5.319148936170213),
  ('R_PPK50r', 5.319148936170222),
  ('R_RnfH2_2', 1000.0),
  ('R_GTPM', 250.0)],
 'R_UTPM': [('R_ATPS4rpp', 250.0),
  ('R_FNOR', -1000.0000000000001),
  ('R_NDPK2', 250.0),
  ('R_SHK3Dr', -1000.0000000000002),
  ('R_SKDH_1', 1000.0000000000002),
  ('R_RnfH2_2', 1000.0000000000001),
  ('R_UTPM', 250.00000000000003)],
 'R_ITPM': [('R_ADK1', 250.0),
  ('R_ADK4', -250.0),
  ('R_ATPS4rpp', 250.0),
  ('R_FNOR', -1000.0),
  ('R_SHK3Dr', -1000.0),
  ('R_SKDH_1', 1000.0),
  ('R_RnfH2_2', 1000.0),
  ('R_ITPM', 250.0)],
 'R_accoaM': [

**These reactions (FNOR and Hnd) are likely only going in this direction**

In [53]:
model.reactions.R_FNOR

R_FNOR: 2.0 M_fdxrd_c + M_h_c + M_nadp_c <-> 2.0 M_fdxo_2_2_c + M_nadph_c

In [54]:
model.reactions.R_FNOR.name

'Ferredoxin---NADP+ reductase'

In [55]:
model.reactions.R_FNOR.reversible=False
model.reactions.R_FNOR.lb=0

In [56]:
results = EGC.EGC_identifier(model)

There ARE energy producing cycles in the model


In [57]:
results

{'R_ATPM': [('R_ATPS4rpp', 250.0),
  ('R_G3PD1ir', 1000.0),
  ('R_G3PD2', 1000.0),
  ('R_ATPM', 250.0),
  ('R_FNRR3_1', 1000.0),
  ('R_Hnd', -1000.0),
  ('R_RnfH2_2', 1000.0)],
 'R_CTPM': [('R_ATPS4rpp', 250.0),
  ('R_GLUDxi', -1000.0),
  ('R_GLUDy', 1000.0),
  ('R_NDPK3', 250.0),
  ('R_FNRR3_1', 1000.0),
  ('R_Hnd', -1000.0),
  ('R_RnfH2_2', 1000.0),
  ('R_CTPM', 250.0)],
 'R_GTPM': [('R_ATPS4rpp', 250.0),
  ('R_G3PD1ir', 1000.0),
  ('R_G3PD2', 1000.0),
  ('R_PPK50_B', 5.319148936170213),
  ('R_PPK50r', 5.319148936170222),
  ('R_FNRR3_1', 1000.0),
  ('R_Hnd', -1000.0),
  ('R_RnfH2_2', 1000.0),
  ('R_GTPM', 250.0)],
 'R_UTPM': [('R_ATPS4rpp', 250.0),
  ('R_G3PD1ir', 1000.0),
  ('R_G3PD2', 1000.0),
  ('R_NDPK2', 250.0),
  ('R_FNRR3_1', 1000.0),
  ('R_Hnd', -1000.0),
  ('R_RnfH2_2', 1000.0),
  ('R_UTPM', 250.0)],
 'R_ITPM': [('R_ADK1', 250.0),
  ('R_ADK4', -250.0),
  ('R_ATPS4rpp', 250.0),
  ('R_G3PD1ir', 1000.0),
  ('R_G3PD2', 1000.0),
  ('R_FNRR3_1', 1000.0),
  ('R_Hnd', -1000.0),
  ('

In [58]:
model.reactions.R_Hnd

R_Hnd: M_h2_c + M_nadp_c <-> M_nadph_c + M_h_c

In [59]:
model.reactions.R_Hnd.name

'hydrogen:NADP+ oxidoreductase'

In [60]:
model.reactions.R_Hnd.reversible=False
model.reactions.R_Hnd.lb=0

In [61]:
results = EGC.EGC_identifier(model)

There are NO energy producing cycles in the model


In [62]:
results

{}

In [63]:
model.update()

In [64]:
constrained_medium_env.apply(model,warning=False,inplace=True)
sol = reframed.pFBA(model)#,constraints={"R_EX_but_e":0})
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.30326536830876416
R_EX_xyl__D_e -10
R_EX_so4_e   -6.23388
R_EX_nh4_e   -3.3848
R_EX_pi_e    -0.299048
R_EX_k_e     -0.0591953
R_EX_cys__L_e -0.01
R_EX_mg2_e   -0.00263083
R_EX_fe3_e   -0.0023679
R_EX_fe2_e   -0.00204022
R_EX_ca2_e   -0.0015785
R_EX_cl_e    -0.0015785
R_EX_h_e      2.71441
R_EX_h2s_e    6.16793
R_EX_ac_e     12.1031
R_EX_co2_e    12.8893
R_EX_h2o_e    21.4258


In [65]:
constrained_medium_env["R_EX_lac__L_e"]=(-10,0)
constrained_medium_env.pop("R_EX_xyl__D_e")

(-10, 0)

In [66]:
constrained_medium_env.apply(model,warning=False,inplace=True)
sol = reframed.pFBA(model)#,constraints={"R_EX_but_e":0})
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.08057991722761619
R_EX_lac__L_e -10
R_EX_h_e     -4.35479
R_EX_so4_e   -1.52316
R_EX_nh4_e   -0.892025
R_EX_pi_e    -0.0794593
R_EX_k_e     -0.0157286
R_EX_cys__L_e -0.01
R_EX_but_e    0.914658
R_EX_h2o_e    1.40318
R_EX_h2s_e    1.51298
R_EX_ac_e     6.96546
R_EX_co2_e    9.0037
R_EX_h2_e     9.85403


In [72]:
constrained_medium_env["R_EX_etoh_e"]=(-10,0)
constrained_medium_env.pop("R_EX_lac__L_e")

(-10, 0)

In [73]:
constrained_medium_env.apply(model,warning=False,inplace=True)
sol = reframed.pFBA(model)#,constraints={"R_EX_but_e":0})
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.0001074239472689649
R_EX_cys__L_e -0.01
R_EX_h2o_e   -0.00830398
R_EX_h_e     -0.00403588
R_EX_so4_e   -0.00117447
R_EX_but_e    0.00117859
R_EX_ac_e     0.00603939
R_EX_co2_e    0.00868509
R_EX_nh4_e    0.00879748
R_EX_h2s_e    0.0111476


In [76]:
constrained_medium_env["R_EX_glyc_e"]=(-10,0)
constrained_medium_env.pop("R_EX_etoh_e")

(-10, 0)

In [77]:
constrained_medium_env.apply(model,warning=False,inplace=True)
sol = reframed.pFBA(model)#,constraints={"R_EX_but_e":0})
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.16349690345146778
R_EX_glyc_e  -10
R_EX_so4_e   -3.71675
R_EX_nh4_e   -1.82021
R_EX_pi_e    -0.161223
R_EX_k_e     -0.0319135
R_EX_cys__L_e -0.01
R_EX_mg2_e   -0.00141834
R_EX_fe3_e   -0.00127658
R_EX_fe2_e   -0.00109993
R_EX_h_e      0.843284
R_EX_but_e    0.924163
R_EX_h2s_e    3.68581
R_EX_ac_e     5.69596
R_EX_co2_e    7.96818
R_EX_h2_e     8.76183
R_EX_h2o_e    13.7993


In [78]:
constrained_medium_env["R_EX_glyald_e"]=(-10,0)
constrained_medium_env.pop("R_EX_glyc_e")

(-10, 0)

In [79]:
constrained_medium_env.apply(model,warning=False,inplace=True)
sol = reframed.pFBA(model)#,constraints={"R_EX_but_e":0})
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.19071004270073835
R_EX_glyald_e -10
R_EX_nh4_e   -2.12484
R_EX_so4_e   -1.60524
R_EX_pi_e    -0.188058
R_EX_k_e     -0.0372253
R_EX_cys__L_e -0.01
R_EX_mg2_e   -0.00165441
R_EX_fe3_e   -0.00148906
R_EX_fe2_e   -0.001283
R_EX_but_e    1.07798
R_EX_h2s_e    1.56748
R_EX_h_e      4.77948
R_EX_ac_e     4.9779
R_EX_h2_e     6.14577
R_EX_h2o_e    6.84335
R_EX_co2_e    7.62833


In [67]:
sol.show_metabolite_balance("M_atp_c",model,abstol=1e-1,equations=True)

[ o <-- ] R_ACKr        6.89393  	M_ac_c + M_atp_c <-> M_actp_c + M_adp_c
[ o <-- ] R_ADK1        0.24961  	M_amp_c + M_atp_c <-> 2.0 M_adp_c
[ o <-- ] R_HMR_0156    0.914658 	M_atp_c + M_but_c + M_coa_c <-> M_amp_c + M_btcoa_c + M_ppi_c
[ o --> ] R_CBPS_1     -0.100981 	2.0 M_atp_c + M_gln__L_c + M_h2o_c + M_hco3_c --> 2.0 M_adp_c + M_cbp_c + M_glu__L_c + 2.0 M_h_c + M_pi_c
[ o --> ] R_GLNS       -0.186888 	M_atp_c + M_glu__L_c + M_nh4_c --> M_adp_c + M_gln__L_c + M_h_c + M_pi_c
[ o --> ] R_PC         -0.249838 	M_atp_c + M_hco3_c + M_pyr_c --> M_adp_c + M_h_c + M_oaa_c + M_pi_c
[ o --> ] R_PGK        -0.282531 	M_3pg_c + M_atp_c <-> M_13dpg_c + M_adp_c
[ o --> ] R_PPDK       -0.459183 	M_atp_c + M_pi_c + M_pyr_c --> M_amp_c + M_h_c + M_pep_c + M_ppi_c
[ o --> ] R_SULabc     -1.52316  	M_atp_c + M_h2o_c + M_so4_e --> M_adp_c + M_h_c + M_pi_c + M_so4_c
[ o --> ] R_Growth     -4.36137  	0.000223 M_10fthf_c + 0.513689 M_ala__L_c + 0.000223 M_amet_c + 0.295792 M_arg__L_c + 0.241055 M_asn_

In [68]:
sol.show_metabolite_balance("M_fdxrd_c",model,abstol=1e-1,equations=True)

[ --> o ] R_POR_syn     17.8787  	M_coa_c + 2.0 M_fdxo_2_2_c + M_pyr_c --> M_accoa_c + M_co2_c + 2.0 M_fdxrd_c + M_h_c
[ --> o ] R_BCD1        1.82932  	M_b2coa_c + 2.0 M_nadh_c + 2.0 M_fdxo_2_2_c <-> M_btcoa_c + 2.0 M_nad_c + 2.0 M_fdxrd_c
[ <-- o ] R_FNRR3_1    -19.7081  	2.0 M_fdxo_2_2_c + M_h2_c <-> 2.0 M_fdxrd_c + 2.0 M_h_c


In [69]:
reframed.FVA(model,reactions=["R_RnfH2_2"])

{'R_RnfH2_2': [0.0, 16.683361111111108]}

### Removing oxygen related reactions

In [70]:
model.update()
model.remove_reactions(model.get_metabolite_reactions("M_o2_e"))
model.remove_metabolite("M_o2_e")
model.update()

### Save models

In [71]:
model.update()

reframed.save_cbmodel(model=model,filename="../output/GEMs/GEMs_final/CH14-bin.0.xml")

## CH14-bin.1 (Lacrimispora)

In [80]:
model_lac = reframed.load_cbmodel("../output/GEMs/GEMs_final/CH14-bin.1.xml")

In [81]:
constrained_medium_env.pop("R_EX_glyald_e")

(-10, 0)

In [82]:
constrained_medium_env.apply(model_lac,warning=False,inplace=True)
sol = reframed.pFBA(model_lac)#,constraints={"R_EX_but_e":0})
print(sol.values["R_Growth"])
sol.show_values(pattern="R_EX_",sort=True,abstol=1e-3)

0.026112947575043
R_EX_h2o_e   -0.676629
R_EX_nh4_e   -0.316847
R_EX_xylan4_e -0.0923077
R_EX_so4_e   -0.0486008
R_EX_xylan8_e -0.0461538
R_EX_pi_e    -0.0257498
R_EX_cys__L_e -0.01
R_EX_k_e     -0.00509706
R_EX_thm_e    0.00863221
R_EX_4hba_e   0.00863803
R_EX_gcald_e  0.0086555
R_EX_h2s_e    0.043424
R_EX_h_e      1.16369
R_EX_ac_e     1.1637
R_EX_co2_e    1.20761
R_EX_h2_e     1.75254


In [85]:
reframed.FVA(model=model_lac,reactions=["R_EX_gua_e"],obj_frac=1)

{'R_EX_gua_e': [0.0, -0.0]}

In [86]:
reframed.FVA(model=model_lac,reactions=["R_EX_gua_e"],obj_frac=0.9)

{'R_EX_gua_e': [0.0, 0.016591278627310205]}

In [87]:
reframed.FVA(model=model_lac,reactions=["R_EX_gua_e"],obj_frac=0.)

{'R_EX_gua_e': [0.0, 0.1643522267206475]}